In [3]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
#!/usr/bin/env python
# pylint: disable=unused-argument
# This program is dedicated to the public domain under the CC0 license.
import nest_asyncio

nest_asyncio.apply()
"""
Simple Bot to reply to Telegram messages.

First, a few handler functions are defined. Then, those functions are passed to
the Application and registered at their respective places.
Then, the bot is started and runs until we press Ctrl-C on the command line.

Usage:
Basic Echobot example, repeats messages.
Press Ctrl-C on the command line or send a signal to the process to stop the
bot.
"""
import os

os.chdir("/home/carnufex/butlerbot")
import logging
from chains import category_chain

from telegram import ForceReply, Update
from telegram.ext import (
    Application,
    CommandHandler,
    ContextTypes,
    MessageHandler,
    filters,
)

# Enable logging
logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO
)
# set higher logging level for httpx to avoid all GET and POST requests being logged
logging.getLogger("httpx").setLevel(logging.WARNING)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context.
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    await update.message.reply_html(
        rf"Hi {user.mention_html()}!",
        reply_markup=ForceReply(selective=True),
    )


async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Send a message when the command /help is issued."""
    await update.message.reply_text("Help!")


async def echo(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Echo the user message."""
    await update.message.reply_text(update.message.text)


async def llm_response(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    await update.message.reply_html(
        rf"Hi {user.mention_html()}!",
        reply_markup=ForceReply(selective=True),
    )
    note = await category_chain.ainvoke(
        {
            "input": update.message.text,
        }
    )
    await update.message.reply_text(note.model_dump_json())


def main() -> None:
    """Start the bot."""
    # Create the Application and pass it your bot's token.
    application = (
        Application.builder()
        .token("7561599888:AAF0uFMJrNRovEZMJC_K09DPbUaQL7uWH5U")
        .build()
    )

    # on different commands - answer in Telegram
    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    application.add_handler(
        MessageHandler(filters.TEXT & ~filters.COMMAND, llm_response)
    )

    # Run the bot until the user presses Ctrl-C
    application.run_polling(allowed_updates=Update.ALL_TYPES)


if __name__ == "__main__":
    main()

In [ ]:
import requests


def get_weather(zipcode: int) -> str:
    """Get the weather for a given zipcode."""
    base_url = "http://api.weatherapi.com/v1"

    try:
        response = requests.get(
            f"{base_url}/forecast.json",
            params={"key": settings.WEATHER_API_KEY, "q": zipcode, "days": 3},
        )
        response.raise_for_status()

        data = response.json()
        forecast = data["forecast"]["forecastday"]

        weather_report = []
        for day in forecast:
            date = day["date"]
            condition = day["day"]["condition"]["text"]
            temp = day["day"]["avgtemp_c"]
            weather_report.append(f"{date}: {condition}, {temp}°C")

        return "\n".join(weather_report)

    except requests.RequestException as e:
        logger.error(f"Weather API error: {str(e)}")
        return "Could not fetch weather data"

In [13]:
from bs4 import BeautifulSoup
from typing import List, Dict
from datetime import datetime
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


class HackerNewsScaper:
    """Simple synchronous scraper for Hacker News articles"""

    def __init__(self):
        self.base_url = "ttps://hacker-news.firebaseio.com"
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
        }

    def get_article_content(self, url: str) -> Dict:
        """Fetch and parse individual article content"""
        try:
            response = requests.get(url, headers=self.headers)
            if response.status_code == 200:
                return {
                    "url": url,
                    "html": response.text,
                    "timestamp": datetime.now().isoformat(),
                }
            logger.error(f"Failed to fetch {url}: Status {response.status_code}")
        except Exception as e:
            logger.error(f"Error fetching {url}: {str(e)}")
        return None

    def get_top_stories(self) -> List[Dict]:
        """Get all articles from HN front page"""
        articles = []
        try:
            # Get the main page
            response = requests.get(self.base_url, headers=self.headers)
            soup = BeautifulSoup(response.text, "html.parser")

            # Find all story links
            story_links = soup.find_all("a", class_="titleline")

            # Process each link
            for link in story_links:
                url = link.get("href")
                if url:
                    # Add base url if needed
                    if url.startswith("item?"):
                        url = f"{self.base_url}/{url}"

                    # Get article content
                    article = self.get_article_content(url)
                    if article:
                        articles.append(article)

        except Exception as e:
            logger.error(f"Error fetching top stories: {str(e)}")

        return articles


# Example usage
scraper = HackerNewsScaper()
articles = scraper.get_top_stories()

# Print results
for i, article in enumerate(articles, 1):
    print(f"\n{i}. {article['url']}")
    print(f"Content length: {len(article['html'])} chars")
    print(f"Fetched at: {article['timestamp']}")

ERROR:__main__:Error fetching top stories: No connection adapters were found for 'ttps://hacker-news.firebaseio.com'


In [11]:
articles

[]

In [ ]:
import os

os.chdir("/home/carnufex/butlerbot")

note = category_chain.invoke(
    {
        "input": "I have a meeting tomorrow at 10 am with the tax man",
    }
)
note

In [ ]:
def llm_response(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    await update.message.reply_html(
        rf"Hi {user.mention_html()}!",
        reply_markup=ForceReply(selective=True),
    )
    note = category_chain.invoke(
        {
            "input": update.message.text,
        }
    )
    await update.message.reply_text(note)

In [ ]:
## turn NOTE into a polars df with same columns as the one in the database
import polars as pl

note.model_dump_json()
notes = pl.DataFrame(note.model_dump())
notes.write_parquet("/home/carnufex/butlerbot/data/notes.parquet", compression="lz4")
notes